In [101]:
import csv
import nltk
from nltk.stem.porter import *
import re
import pandas as pd

In [102]:
#open the file with the russian tweets text
df_xlsx = pd.read_excel('/Volumes/Katja/Katja/Hate_Speech_projekt/russische Texte.xlsx')

In [105]:
#convert the df_xlsx-file to the df-csv file
df_xlsx.to_csv("russian_texts.csv")

In [106]:
df = pd.read_csv("/Volumes/Katja/Katja/Hate_Speech_projekt/russian_texts.csv")

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [107]:
print(df.head(5))

   Unnamed: 0  Unnamed: 0.1             Tweet_ID Language  \
0           0             0  1391211609452843008       ru   
1           1             1  1391211683889168396       ru   
2           2             2  1391212119316738050       ru   
3           3             3  1391212598943694848       ru   
4           4             4  1391212890858864645       ru   

                                                text Geolocation_coordinate  \
0  После вакцинации AstraZeneca экс-чемпиону мира...                     NO   
1  COVID-19 в Испании: страна отменила режим чрез...                     NO   
2  Назван вес, при котором увеличивается риск гос...                     NO   
3  Коронавирус в Казахстане: +2350 новых случаев ...                     NO   
4  Коронавирус в Казахстане: +2350 новых случаев ...                     NO   

   RT  Likes  Retweets Country                    Date Created  
0  NO      0         0     NaN  Sun May 09 02:01:09 +0000 2021  
1  NO      0         0     N

In [133]:
#implement stop_words from nltk
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

In [134]:
other_exclusions = ['#ff',"ff","rt"]
russian_stopwords.extend(other_exclusions)

In [135]:
#implement PorterStemmer from nltk for russian language
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")


In [136]:
#extract all links from the texts
def preprocess(text_string):
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', str(text_string))
    parsed_text = re.sub(giant_url_regex, '', str(parsed_text))
    parsed_text = re.sub(mention_regex, '', str(parsed_text))
    return str(parsed_text)

In [137]:
tweets_text =df.text.to_list()

In [138]:
tweets_text_preprocess = list(map(preprocess, tweets_text))
print(tweets_text[5:10])

['Врачи оценили риск повторно заразиться COVID-19 при высоком титре антител https://t.co/NflgxtOYRr', 'RT @tegsh78: @MdyBolor @Miigaa_d @GanzoTg @OyunerdeneMN @AmarsaikhanS @BattulgaKh Харин сонин короно байх юм аа Ардын намын ойд очихгүй мөр…', 'RT @ikonnews: Энэтхэгт нэг хоногийн дотор анх удаа 4,000 гаруй хүн Covid-19 өвчнөөр нас барав https://t.co/MLf0kyo0Kt https://t.co/UGKOKWVq…', 'RT @tegsh78: @MdyBolor @Miigaa_d @GanzoTg @OyunerdeneMN @AmarsaikhanS @BattulgaKh Харин сонин короно байх юм аа Ардын намын ойд очихгүй мөр…', 'Решение властей подарить ветеранам одеяла, снижение платы за тепло на 46% и вспышка COVID-19 среди студентов БГМУ: Главные новости недели в Башкирии \nРедакция https://t.co/mhPHxFzpQm подготовила очередной обзор главных новостей уходящей недели в Баш... https://t.co/2ZgvnPtV5t https://t.co/CVyM8E0JH5']


In [139]:
#tokenize the tweets
def tokenize(tweet):
    tweet = " ".join(re.split("[^\w]", tweet.lower())).strip()
    tokens = [stemmer.stem(t) for t in tweet.split()]
    return tokens


#[\p{IsCyrillic}][0-9\p{IsCyrillic}]+

In [140]:
#tokenize_tweets_text = list(map(tokenize, tweets_text_preprocess))
#print(tweets_text_preprocess[5:10])

['Врачи оценили риск повторно заразиться COVID-19 при высоком титре антител ', 'RT :       Харин сонин короно байх юм аа Ардын намын ойд очихгүй мөр…', 'RT : Энэтхэгт нэг хоногийн дотор анх удаа 4,000 гаруй хүн Covid-19 өвчнөөр нас барав  …', 'RT :       Харин сонин короно байх юм аа Ардын намын ойд очихгүй мөр…', 'Решение властей подарить ветеранам одеяла, снижение платы за тепло на 46% и вспышка COVID-19 среди студентов БГМУ: Главные новости недели в Башкирии Редакция  подготовила очередной обзор главных новостей уходящей недели в Баш...  ']


In [147]:
#get the tokens from the texts f.ex. I am eating --> ["i","to be","to eat"]
from pymystem3 import Mystem
mystem = Mystem()
from string import punctuation
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
             and token !=" "\
             and token.strip() not in punctuation]
    text = " ".join(tokens)
    return text


In [148]:
tokenize_tweets_text = list(map(preprocess_text, tweets_text_preprocess))

In [149]:
print(tokenize_tweets_text[5:10])

['врач оценивать риск повторно заражаться covid 19 высокий титр антитело', 'харин сонин короно байх юм аа ардын намын ойд очихгүй мөр …', 'энэтхэгт нэг хоногийн дотор анх удаа 4 000 гаровать хүн covid 19 өвчнөөр барывать …', 'харин сонин короно байх юм аа ардын намын ойд очихгүй мөр …', 'решение власть подарить ветеран одеяло снижение плата тепло 46 вспышка covid 19 среди студент бгма главный новость неделя башкирия редакция подготовить очередной обзор главный новость уходить неделя баш ...']


In [157]:
from pymorphy2 import MorphAnalyzer
import pymorphy2
import re


In [222]:
#tried to extract also languages as kasachich, which have 
#the same letters as russian languages, but which I dont speak and dont understand
# tweet that has 1.0 point or approximately 1.0 belong to russian language, the other not
#but this code doesnt work 
m = MorphAnalyzer()
r = re.compile(r'\w\+')
only_russian_tweets = []
for tweet in tokenize_tweets_text:
    cnt = 0
    bad = 0
    for word in r.findall(tweet):
        cnt += 1
        p = m.parse(word)[0]
        bad += (not p.tag.POS or \
               isinstance(p.methods_stack[0][0],pymorphy2.units.by_analogy.KnownSuffixAnalyzer.FakeDictionary))
    try:
        if bad/cnt < 0.25:
            only_russian_tweets.append(tweet)
    except ZeroDivisionError: 
        continue
        #print(f'{bad/cnt:0.2f}', tweet)


In [223]:
only_russian_tweets[5:21]

[]

In [209]:

#rus = set('Ә ә, Ө ө, Ү ү, Ғ ғ, Қ қ. Ң ң, Ү́ ү́, Ұ ұ ')
#extract other tweets that have other letters than russian language and english (because tweets were 
#collect by "covid"-hashtags)


rus = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюяabcdefghijklmnopqrstuvwxyz0123456789 ')
conv_text = lambda mas_in : [''.join([j for j in i if j.lower() in rus]) for i in mas_in]





In [210]:
conv_text(only_russian_tweets[5:30])

['врач оценивать риск повторно заражаться covid 19 высокий титр антитело',
 'харин сонин короно байх юм аа ардын намын ойд очихгй мр ',
 'энэтхэгт нэг хоногийн дотор анх удаа 4 000 гаровать хн covid 19 вчнр барывать ',
 'харин сонин короно байх юм аа ардын намын ойд очихгй мр ',
 'решение власть подарить ветеран одеяло снижение плата тепло 46 вспышка covid 19 среди студент бгма главный новость неделя башкирия редакция подготовить очередной обзор главный новость уходить неделя баш ',
 'харин сонин короно байх юм аа ардын намын ойд очихгй мр ',
 '2 350 новый факт заражение covid 19 выявлять казахстан сутки',
 'коронавирус россия sars cov 2 знать праздник    весь новость',
 'смертность covid 19 мир выше 2 исследователь институт показатель оценка здоровье медицинский школа вашингтонский университет сша institute for health metrics and evaluation ihme считать ',
 'nan',
 'дэмба sinopharm covid 19 вакциныга яаралтай горимд хэрэглэхийг звшрсн жагсаалтад бртгэлээ',
 'воз мир смочь достигать ко